In [70]:
import os
import sys
import re
import numpy as np

from ReadMaestro import maestro
from ReadMaestro import parse_pursuit_trials
paths = [p for p in sys.path if os.path.exists(p)]
print(paths)

from importlib import reload
reload(maestro)
reload(parse_pursuit_trials)

['/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '/home/seth/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']


<module 'ReadMaestro.parse_pursuit_trials' from '/home/seth/analysis/ReadMaestro/ReadMaestro/parse_pursuit_trials.py'>

In [2]:
file_path = '/home/seth/Projects/DynamicCoherencePhysiology/ar/20220404a/ar220404a.0001'
with open(file_path, 'rb') as f:
    data_file = maestro.DataFile.load(f.read(), file_path)

In [72]:
data = maestro.load_directory('/home/seth/Projects/DynamicCoherencePhysiology/ar/20220404a')

In [73]:
purs = parse_pursuit_trials.pursuitDataObject
purs.setName(purs,data)
print(purs.name)

purs.pursuitData(purs,data,trialIDs=['dPref'], directions=[0.0, 45.0, 90.0, 135.0, 180.0, 225.0, 270.0, 315.0], speeds=[10.0], coherences=[], perturbations=[])
print(purs.hvelocities)
print(purs.saccades)

/home/seth/Projects/DynamicCoherencePhysiology/ar/2022040
[[-1.10268 -1.10268 -0.55134 ...  0.09189 -0.45945  0.     ]
 [ 0.27567  0.45945  0.      ... -0.09189  0.64323  0.     ]
 [ 0.73512  0.       0.18378 ...  0.27567 -0.36756  0.     ]
 ...
 [-0.09189  0.27567 -0.09189 ... -0.18378  0.09189 -0.09189]
 [ 0.09189 -0.18378  0.09189 ...  0.18378  0.09189 -0.09189]
 [ 0.55134  0.09189  0.36756 ...  0.18378  0.36756 -0.36756]]
[[     nan      nan -0.55134 ...  0.09189 -0.45945  0.     ]
 [     nan      nan -0.55134 ...  0.       0.18378  0.     ]
 [     nan      nan -0.36756 ...  0.27567 -0.18378  0.     ]
 ...
 [ 0.       0.36756 -0.09189 ... -1.19457  0.18378  0.36756]
 [ 0.09189  0.18378  0.      ... -1.01079  0.27567  0.27567]
 [ 0.64323  0.27567  0.36756 ... -0.82701  0.64323 -0.09189]]
[[ True  True False ... False False False]
 [ True  True False ... False False False]
 [ True  True False ... False False False]
 ...
 [False False False ... False False False]
 [False False False .

In [74]:
np.array(data[0].ai_data[2][0:10])*0.09189


array([-1.10268, -0.82701, -0.09189, -0.18378, -0.9189 , -0.82701,
       -0.27567,  0.     ,  0.     ,  0.     ])

(1751, 1)
[90.0]


[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]
[[0.5 0.  0.5]
 [1.  0.5 1. ]
 [1.  1.  1. ]
 [1.  1.  1. ]]
[[ True False  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]
[[1. 0. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
